# Các thư viện được sử dụng

In [1]:
from sympy import *
import numpy as np

# Nhập vào dữ liệu

In [2]:
t = symbols('t')

# Nhập hàm theo biến t
function = sin(t)*cos(t)

# khoảng tích phân
A = 0
B = 1

# số cấp đa thức xấp xỉ
N = 10

# Một số hàm phụ trợ

## Phân tích đa thức $\prod_{i=0,i\neq k}^n (x-i)$

### Input:
        Cấp n của đa thức
        Số k bị khuyết
### Output:
        Mảng các hệ số sau khai triển
### Thuật toán:
<center><img src="khaitriendathuckhuyet.png"><center>

In [3]:
def factorize_poly(n: int, k: int) -> np.array:
    coefs = np.zeros([n, n+2])

    coefs[0, n-1], coefs[0, n] = 1, -1
    for i in range(1, n):
        coefs[i, n-i-1] = 1
        for j in range(n-i, n+1):
            coefs[i, j] = coefs[i-1, j+1] - (i+1)*coefs[i-1, j]
    
    n_poly = coefs[n-1]
    res = np.zeros(n+1)
    res[0] = 1
    for i in range(1, n+1):
        res[i] = res[i-1]*k + n_poly[i]
    return res

## Tính giá trị của đa thức $\sum_{i = 1}^n a_i.x^i$

### Input:
        Mảng arr các hệ số đa thức (bậc n -> bậc 1)
        Điểm x
### Output:
        Giá trị tại điểm x
### Thuật toán:
<center><img src="tinhgiatridathuc.png"><center>

In [4]:
def Horner(arr: np.array, x: int):
    temp = np.zeros(len(arr)+1)
    temp[0] = arr[0]
    for i in range(1, len(arr)):
        temp[i] = temp[i-1]*x + arr[i]
    temp[-1] = temp[len(arr)-1]*x
    return temp[-1]

# Tìm các hệ số Cotes

Ta biết, 
$$\begin{align}
\int_a^b f(x)dx \approx \int_a^b P_n(x_0+t.h)dx
&= h.\int_0^n P_n(t)dt \\ 
&= h.\sum_{i=0}^n A_i.y_i 
\end{align}$$
Theo Lagrange, $ A_i ={\displaystyle \int_0^n \prod_{k\neq i} \frac{t-k}{i-k}dt} $

Đặt tử số: nume = $\int_0^n \prod_{k\neq i} (t-k)$

Đặt mẫu số: deno = $\prod_{k\neq i} (i-k)$

## Tính tích phân các đa thức trên tử (tìm nume)

### Input:
        Cấp n của đa thức
### Output:
        Mảng các hệ số sau tích phân
### Thuật toán:
<center><img src="tinhhesotrentu.png"><center>

In [5]:
def numeCoeffs(n: int) -> np.array:
    vacantMatrix = np.zeros([n+1, n+1])
    res = np.zeros(n+1)
    for i in range(n+1):
        vacantMatrix[i] = factorize_poly(n, i)
        for j in range(n+1):
            vacantMatrix[i][j] = 1/(n+1-j) * vacantMatrix[i][j]
        for k in range(n+1):
            res[k] = Horner(vacantMatrix[k], n)
    return res

## Tính các hệ số dưới mẫu (tìm deno)

### Input:
        Cấp n của đa thức
### Output:
        Mảng các hệ số tích
### Thuật toán
<center><img src="timhesoduoimau.png"><center>

In [6]:
def denoCoeffs(n: int) -> np.array:
    res = np.ones(n+1)
    for i in range(n+1):
        for j in range(n+1):
            if (j != i):
                res[i] *= (i - j)
    return res

# Khai triển thuật toán Newton-Cotes

### Input:
        Hàm số f theo biến t
        Khoảng tích phân (a, b)
        Cấp n của đa thức xấp xỉ
### Output:
        Giá trị tích phân
### Thuật toán:
<center><img src="khaitriennewtoncotes.png"><center>

In [7]:
def my_integrate(func, a, b, n):
    nume = numeCoeffs(n)
    deno = denoCoeffs(n)
    CotesCoeffs = np.divide(nume, deno) / n

    h = (b-a) / n   # step
    res = 0
    xn = a
    for c in CotesCoeffs:
        res += func.subs(t, xn) * c
        xn  += h
    res *= (b-a)
    return res

In [8]:
integral = my_integrate(function, A, B, N)
print(integral)
# do something

0.354036709154479
